In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras
import h5py
import numpy as np
from Utility import Utility

/home/albert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 3 layer LSTM

## Read in sonnets and sequence them into integer sequences via map

In [2]:
util = Utility()
sonnets = util.get_shakespeare()

In [3]:
sonnetString = []
for i in range(len(sonnets)):
    sA = sonnets[i+1]
    sonnetString.append("")
    for k in sA:
        for j in k:
            sonnetString[-1] += j + " "

In [4]:
#LUT
int_to_char = {0: ' ', 1: "'", 2: ',', 3: '-', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'y', 27: 'j', 28: 'x', 29: 'z'}
char_to_int = {' ': 0, "'": 1, ',': 2, '-': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'y': 26, 'j':27, 'x':28, 'z':29}

In [5]:
window = 40
X = []
Y = []
dataX_seq = []
for iter in range(len(sonnetString)):
    text = sonnetString[iter]
    seq = text[0:window]
    dataX_seq.append([char_to_int[char] for char in seq])
    for i in range(0, len(text) - window, 1):
        seq = text[i:i + window]
        out = text[i + window]
        X.append([char_to_int[char] for char in seq])
        Y.append(char_to_int[out])

In [6]:
X = np.reshape(X, (len(X), window, 1))
Y = np_utils.to_categorical(Y)

## Make model and train (takes forever...)

In [7]:
model = Sequential()
model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dense(Y.shape[1], activation='softmax'))
adam = keras.optimizers.Adam(lr = 1e-4)
model.load_weights('weights-improvement-512230.hdf5')
model.compile(loss='categorical_crossentropy', optimizer=adam)
#filename = "weights-improvement-61-1.4022.hdf5"
#model.load_weights(filename)

In [1]:
#taken from some guy on stackoverflow - calculates how much vram the model needs

def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

In [33]:
get_model_memory_usage(400, model)

0.82

In [54]:
#checkpoint code taken from public tutorial
filepath="singlelayer2-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, Y, epochs=100, batch_size=300, callbacks=callbacks_list)

Epoch 1/100
86726/86726 [==============================] - 51s 586us/step - loss: 0.4589

Epoch 00001: loss improved from inf to 0.45893, saving model to singlelayer2-01.hdf5
Epoch 2/100
86726/86726 [==============================] - 50s 576us/step - loss: 0.4611

Epoch 00002: loss did not improve
Epoch 3/100
86726/86726 [==============================] - 51s 587us/step - loss: 0.4584

Epoch 00003: loss improved from 0.45893 to 0.45839, saving model to singlelayer2-03.hdf5
Epoch 4/100
86726/86726 [==============================] - 50s 575us/step - loss: 0.4557

Epoch 00004: loss improved from 0.45839 to 0.45572, saving model to singlelayer2-04.hdf5
Epoch 5/100
86726/86726 [==============================] - 49s 560us/step - loss: 0.4555

Epoch 00005: loss improved from 0.45572 to 0.45552, saving model to singlelayer2-05.hdf5
Epoch 6/100
86726/86726 [==============================] - 48s 557us/step - loss: 0.4585

Epoch 00006: loss did not improve
Epoch 7/100
86726/86726 [===============

KeyboardInterrupt: 

## Predictions

In [8]:
#sample function from keras example code (see keras official github)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds+0.001) / temperature
    exp_preds = np.exp(preds)
    if(np.sum(exp_preds) != 0):
        preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
seed = '  shall i compare thee to a summer\'s day'
seedArr = []
for i in range(len(seed)):
    seedArr.append(char_to_int[seed[i]])

In [10]:
# pick a random seed that is the actual start of a sonnet
start = np.random.randint(0, len(dataX_seq)-1)
pattern = dataX_seq[start]
pattern = seedArr
print(''.join([int_to_char[nt] for nt in pattern]))

  shall i compare thee to a summer's day


In [13]:
# generate characters
nchar = 1400
for i in range(nchar):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)[0]
    index = sample(prediction, 0.25)
    
    result = int_to_char[index]
    print(result, end='')
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

trength of love, with in the eyes of thee this i prognostinate thee that thou art not, though i feel thou wealth she had in this, my lovd no whit disdarte, living on a living brow before the golden tresses of the dead, the right of sepulchres, were shorn away, to live a second life on second head, ere beauty's dead fleece made another gay in him those holy antique hours are seen, without all ornament, it self and true, making no summer of another's green, robbing no old to dress his beauty new, and him as for a map doth nature store, to show false art what beauty was of yore being ahe the ceasth's eear from my love's time these would i say my self the paintered seem from thee, thou shall have profaned all wormd with travels the cloud thou break, to dry the rain on my storm-beaten face, for no man well of such a salve can speak, that heals the wound, and cures not the disgrace nor can thy shame give physic to my grief, though thou repent, yet i have still the loss, th' offender's sorrow